# Métodos de iteración funcional para sistemas de ecuaciones

Entradas:

 - una función $g: \mathbb{R}^N \to \mathbb{R}^N, g\in \mathscr{C}^1(\mathbb{R}^N)$,
 - $x_0 \in \mathbb{R}^N$.
 
y, según la variante:
 
  1. tolerancia entre dos iteraciones consecutivas,
  2. solución y tolerancia,
  3. constante de Lipstchiz y tolerancia.

Primero, definimos un generador de los términos de la sucesión para cada método.

In [1]:
def functional_iteration(g, x_0):
    
    x = vector(x_0)
    while True:
        yield x
        x = g(*x).n()

En el método de Newton, resolvemos el SEL correspondiente en lugar de hallar la inversa de la matriz.

In [2]:
def newton(g, x_0):
    x = vector(x_0)
    f = g.derivative()
    while True:
        yield x
        y = f(*x).solve_right(g(*x))
        x = vector(x) - vector(y)

Para la primera variante, creamos un generador que, a partir de uno de los métodos, nos dé la distancia entre dos aproximaciones consecutivas, y nos quedamos con el primer elemento del generador para el que la distancia con la aproximación anterior sea menor que la tolerancia dada.

In [3]:
def adjacent_distance(method, g, x_0):
    gen = method(g, x_0)
    x = gen.next()
    
    for i in gen:
        yield (i,(x-i).norm(oo))
        x = i

In [4]:
import itertools as it

def first_variant(method, g, x_0, epsilon):
    orig_gen = adjacent_distance(method,g,x_0)
    gen = it.takewhile(lambda (x,e) : e >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

Para la segunda variante, procedemos de forma similar y creamos un generador que pare en la primera aproximación que nos proporcione un error menor que la tolerancia.

In [5]:
def second_variant(method, g, x_0, sol, epsilon):
    orig_gen = method(g,x_0)
    gen = it.takewhile(lambda (x) : abs(x-sol) >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

En este caso, tenemos $L\in (L_0,1)$, con $L_0$ la constante de Lipschitz de $g$, y el teorema del punto fijo nos da la cota

$$|x_n-x^*| \le \frac{L^n}{1-L}|g(x_0)-x_0|$$

Ahora, si imponemos 

$$\frac{L^n}{1-L}|g(x_0)-x_0| < \varepsilon$$

tenemos que

$$ n > \frac{\log\left(\displaystyle\frac{\varepsilon(1-L)}{|g(x_0)-x_0|}\right)}{\log(L)} $$

In [6]:
def third_variant(g, x_0, L, epsilon):
    n_iterations = int((log(epsilon*(1-L)/abs(vector(g(*x_0))-vector(x_0)))/log(L)))+2
    
    print("Iterations: {}".format(n_iterations))
    
    return it.islice(functional_iteration(g,x_0), n_iterations)

In [7]:
def print_all(gen):
    for i in gen:
        print(i)
        
def print_last(gen):
    j = None
    for j in gen:
        pass
    print(j)

## 2: ejemplos.

Tomamos la función:
$$
    \begin{cases}
    f_1(x,y) = y- arctg(x)\\
    f_2(x,y) = x - arctg(y)
    \end{cases}
$$

Así, podemos definir dos formas de aplicar la iteración funcional.
La primera sería $g_1$:

$$
    \begin{cases}
    y=arctg(x) \\
    x = arctg(x)
    \end{cases}
$$

Y la segunda sería $g_2$:
$$
    \begin{cases}
    y=tg(x) \\
    x = tg(y)
    \end{cases}
$$

La tangente es claramente no contractiva, por lo que escogeremos ahora el dominio para la función $g_1$ que será: $D= (-\frac{\pi}{2}, \frac{\pi}{2})$. En este dominio, $g_1(D) \subseteq D$.

Ahora, analizamos lo que se ha pedido:

In [8]:
import sys

f(x,y) = (y-arctan(x), x-arctan(y))

g_1(x,y) = (arctan(y), arctan(x))
g_2(x,y) = (tan(x), tan(y))

print("Todos las aproximaciones del primer método, primera variante")
gen = first_variant(functional_iteration, g_1, (0.5,.5), 10e-4)

#for fix_point, error in gen:
#    print("Aproximación: {}, distancia entre términos: {}".format(fix_point, error))


print("Algunas iteraciones del segundo método")
gen = first_variant(functional_iteration, g_2, (1,1), 10e-8)

#for fix_point, error in it.islice(gen,1,90):
#    print("Aproximación: {}, distancia entre término: {}".format(fix_point, error))


print("Todos las aproximaciones del primer método, segunda variante")
gen = second_variant(functional_iteration, g_1, (0.5,.5), vector([0, 0]), 10e-2)

#for fix_point in gen:
#    print("Aproximación: {}, error: {}".format(fix_point, fix_point.norm()))


print("Todos las aproximaciones del primer método, tercera variante")
gen = third_variant(g_1, (0.5,.5), 0.9999, 10e-1)

#for fix_point in gen:
#    print("Aproximación: {}, error: {}".format(fix_point, fix_point.norm()))

print("Aproximacion de la solución por el metodo de Newton.")
gen = first_variant(newton, f, (0.5,0.5), 10e-4)
    
for fix_point, error in gen:
    print("Aproximación: {}, error: {}".format(fix_point, error))

Todos las aproximaciones del primer método, primera variante
Algunas iteraciones del segundo método
Todos las aproximaciones del primer método, segunda variante
Todos las aproximaciones del primer método, tercera variante
Iterations: 62423
Aproximacion de la solución por el metodo de Newton.
Aproximación: (0.318238045004030, 0.318238045004030), error: 0.181761954995970
Aproximación: (0.208038059631220, 0.208038059631220), error: 0.110199985372810
Aproximación: (0.137513272689158, 0.137513272689158), error: 0.0705247869420625
Aproximación: (0.0913315808523198, 0.0913315808523198), error: 0.0461816918368380
Aproximación: (0.0607865035595681, 0.0607865035595681), error: 0.0305450772927517
Aproximación: (0.0404944355571086, 0.0404944355571086), error: 0.0202920680024595
Aproximación: (0.0269874428881970, 0.0269874428881970), error: 0.0135069926689116
Aproximación: (0.0179890086697816, 0.0179890086697816), error: 0.00899843421841546
Aproximación: (0.0119918963777518, 0.0119918963777518), er

In [9]:
contractive2(x,y) = (0.5*x+3, 0.5*y)

print("Todos las aproximaciones de la primera variante")
gen = first_variant(newton, contractive2, (0.5,.5), 0.0000005)

fix_point, error = None, None
for fix_point,error in gen:
    print("Aproximación: {}, distancia entre términos: {}".format(fix_point, error))
    
print("Todos las aproximaciones de la tercera variante")
gen = third_variant(contractive2, (5,5), 0.8, 10e-9)

fix_point = None, None
for fix_point in gen:
    print("Aproximación: {}, error: {}".format(fix_point, abs(vector((6,0))-fix_point)))

print("")
print("Última aproximación de la segunda variante")
print_last(second_variant(functional_iteration, contractive2, (5,5), fix_point, 10e-9))

Todos las aproximaciones de la primera variante
Aproximación: (-6.00000000000000, 0.000000000000000), distancia entre términos: 6.50000000000000
Aproximación: (-6.00000000000000, 0.000000000000000), distancia entre términos: 0.000000000000000
Todos las aproximaciones de la tercera variante
Iterations: 95
Aproximación: (5, 5), error: sqrt(26)
Aproximación: (5.50000000000000, 2.50000000000000), error: 2.54950975679639
Aproximación: (5.75000000000000, 1.25000000000000), error: 1.27475487839820
Aproximación: (5.87500000000000, 0.625000000000000), error: 0.637377439199098
Aproximación: (5.93750000000000, 0.312500000000000), error: 0.318688719599549
Aproximación: (5.96875000000000, 0.156250000000000), error: 0.159344359799775
Aproximación: (5.98437500000000, 0.0781250000000000), error: 0.0796721798998873
Aproximación: (5.99218750000000, 0.0390625000000000), error: 0.0398360899499436
Aproximación: (5.99609375000000, 0.0195312500000000), error: 0.0199180449749718
Aproximación: (5.9980468750000

Empecemos con el primer apartado del ejercicio 21 en el que compararemos las ejecuciones y resultados del ejercicio 18, en este se nos pedía resolverlo con el método de convergencia funcional. Ahora nos piden resolverlo con el método de Newton y que comparemos la eficiencia y resultados de ambos métodos.

In [10]:
ejercicio18IterSaez(x,y,z)=((cos(y*z)+0.5)/3,sqrt(x^2-0.81+1.06-16.2*y+sin(z))/9,(-exp(-x*y)-(10*3.14159-3)/3)/20)
ejercicio18IterSofia(x,y,z)=((cos(y*z)+1/2)/3, sqrt((sin(z)+x^2+1.06)/81)-0.1, -(exp(-x*y)+(10*pi.n()-3)/3)/20)
ejercicio18New(x,y,z)=(-cos(y*z)+3*x-0.5,sin(z)+x^2-81*(y+1.06), e^(-x*y)+20*z+(10*pi.n()-3)/3)
print("Todos las aproximaciones de la primera variante con el método de Newton")
Primera = first_variant(newton, ejercicio18New, (0.1,0.1,-0.1), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Primera):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}".format(indice+1,fix_point, ejercicio18New(*fix_point), error))
    

    
print("\n\nTodos las aproximaciones de la primera variante con el método de Iteración Funcional")
Primera = first_variant(functional_iteration, ejercicio18IterSofia, (0.1,0.1,-0.1), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Primera):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}\n".format(indice+1, fix_point, ejercicio18IterSofia(*fix_point), error))
    

Todos las aproximaciones de la primera variante con el método de Newton
1: 
	Aproximación: (0.500229487989596, -1.06536473740976, -0.526888877418178), 
	punto: (0.154139957224511, 0.181926670418692, 0.638110088389407) 
	distancia entre términos: 1.16536473740976
2: 
	Aproximación: (0.443912958344123, -1.06409966530846, -0.553628932088287), 
	punto: (0.000306898293946922, 0.00335407050575973, 0.00318374637341279) 
	distancia entre términos: 0.0563165296454727
3: 
	Aproximación: (0.443785641659663, -1.06406119576418, -0.553775886140014), 
	punto: (4.44298420276823e-9, 2.18872009405402e-8, 2.65153374812144e-8) 
	distancia entre términos: 0.000146954051726822
4: 
	Aproximación: (0.443785639971961, -1.06406119552478, -0.553775887313279), 
	punto: (0.000000000000000, -6.30051566474776e-15, 1.77635683940025e-15) 
	distancia entre términos: 1.11022302462516e-16


Todos las aproximaciones de la primera variante con el método de Iteración Funcional
1: 
	Aproximación: (0.499983333472222, 0.009441

Como podemos ver, con el método de Newton hemos necesitado una iteración menos para alcanzar la tolerancia deseada dentro del contexto de la distancia entre dos términos. No sólo eso, sino que, con una iteración menos hemos conseguido que la distancia entre términos sea 10^8 veces menor. 

Por ello podemos asegurar que, en este caso particular, el método de Newton converge más rápidamente.

A continuación haremos la misma prueba, pero esta vez con el ejercicio 20.

In [12]:
ejercicio20Iter(x,y)=(y/sqrt(5),(sin(x)+cos(y))/4)
ejercicio20New(x,y)=(5*x^2-y^2,y-(sin(x)+cos(y))/4)

print("\n\nTodos las aproximaciones de la primera variante con el método de Iteración Funcional")
Segunda = first_variant(functional_iteration, ejercicio20Iter, (1/4,1/4), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Segunda):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}\n".format(indice+1, fix_point, ejercicio20Iter(*fix_point), error))




Todos las aproximaciones de la primera variante con el método de Iteración Funcional
1: 
	Aproximación: (0.111803398874989, 0.304079095241292), 
	punto: (0.0608158190482584*sqrt(5), 0.266423427535425) 
	distancia entre términos: 0.138196601125011

2: 
	Aproximación: (0.135988305499232, 0.266423427535425), 
	punto: (0.0532846855070850*sqrt(5), 0.275072068196639) 
	distancia entre términos: 0.0376556677058668

3: 
	Aproximación: (0.119148178953540, 0.275072068196639), 
	punto: (0.0550144136393278*sqrt(5), 0.270318023474851) 
	distancia entre términos: 0.0168401265456923

4: 
	Aproximación: (0.123015968639829, 0.270318023474851), 
	punto: (0.0540636046949702*sqrt(5), 0.271597989694383) 
	distancia entre términos: 0.00475404472178792

5: 
	Aproximación: (0.120889895206630, 0.271597989694383), 
	punto: (0.0543195979388767*sqrt(5), 0.270984771837357) 
	distancia entre términos: 0.00212607343319837

6: 
	Aproximación: (0.121462313501786, 0.270984771837357), 
	punto: (0.0541969543674714*sqrt

In [ ]:
print("Todos las aproximaciones de la primera variante con el método de Newton")
Segunda = first_variant(newton, ejercicio20New, (1/4,1/4), 10^(-5))

fix_point, error = None, None
for indice,(fix_point,error) in enumerate(Segunda):
    print("{}: \n\tAproximación: {}, \n\tpunto: {} \n\tdistancia entre términos: {}".format(indice+1,fix_point, ejercicio20New(*fix_point), error))
    


Como podemos ver, en este caso el coste computacional es demasiado grande en el caso del método de Newton. Sin embargo, en el método de iteración funcional el coste computacional es mínimo en comparación.

Tras hacer ambas comparaciones con ambos ejercicios, vemos que, conforme están actualmente diseñados estos algoritmos, el algoritmo de iteración funcional es mucho más eficiente mientras que el de Newton necesita menos iteraciones para converger más rápido.